In [1]:
#import dependencies and setup
import pandas as pd
import os
import numpy as np

school_data_to_load = os.path.join(".", "Resources", "schools_complete.csv")
student_data_to_load = os.path.join(".", "Resources", "students_complete.csv")

In [2]:
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Data

In [3]:
prefixes_suffixes = ["Dr.", "Mr.", "Mrs.", "Miss", "MD", "DDS", "DVM", "Phd"]

for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word, "")

# Replace the reading and math scores of Thomas HS 9th graders with NaN

In [4]:
# Replace reading scores
student_data_df.loc[
    (student_data_df["school_name"] == "Thomas High School")
    & (student_data_df["grade"] == "9th")
    & (student_data_df["reading_score"] > 0),
    "reading_score"]= np.nan

In [5]:
# Replace math scores
student_data_df.loc[
    (student_data_df["school_name"] == "Thomas High School")
    & (student_data_df["grade"] == "9th")
    & (student_data_df["math_score"] > 0),
    "math_score"]= np.nan

# District Summary: Merge Tables and Calculate Attributes

In [6]:
# Left Merge the Students and Schools Data Frames
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

In [7]:
# Define/calculate attributes pt. 1 (school/student counts)
school_count = school_data_complete_df["school_name"].count()
student_count = school_data_complete_df["Student ID"].count()

In [8]:
# Define/calculate attributes pt. 2 (total budget/avg reading/math scores)
total_budget = school_data_df["budget"].sum()
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [9]:
# Define/calculate attributes pt. 3 (passing reading/math count/percentage)
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_percentage = passing_reading_count/ float(student_count) * 100

In [10]:
# Defing/calc attributes pt. 4 (overall passing count/percentage)
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                              &(school_data_complete_df["reading_score"] >= 70)]
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
overall_passing_percentage = overall_passing_math_reading_count / float(student_count) * 100

# District Summary: Format

In [11]:
# Define attribute display names
district_summary_df = pd.DataFrame(
{
    "Total Schools" : [school_count],
    "Total Students" : [student_count],
    "Total Budget" : [total_budget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "% Passing Math" : [passing_math_percentage],
    "% Passing Reading" : [passing_reading_percentage],
    "% Passing Overall" : [overall_passing_percentage]
})

In [12]:
# Format data
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Passing Overall"] = district_summary_df["% Passing Overall"].map("{:.1f}".format)

# Output District Summary
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,39170,"39,170","$24,649,428.00",78.9,81.9,73.9,84.7,64.1


# District Summary: Analsysis

In [13]:
# As a result of nulling-out the 9th grade Thomas HS Math and Science scores
# -Average Math Score drop by 0.1
# -Avearge Reading Score remains the same
# -% Passing Math drops by 1.1%
# -% Passing Reading drops by 1.3%
# -% Passing Overall drops by 0.9%

# School Summary: Attributes

In [14]:
# School type/count/budget/budget per student
per_school_type = school_data_df.set_index(["school_name"])["type"]
per_school_count = school_data_complete_df["school_name"].value_counts()
per_school_budget = school_data_df.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_count

In [15]:
# Avg school math/reading score and school math/reading passing rate pt. 1
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

In [16]:
# School math/reading passing rate pt. 2
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

In [17]:
# School math/reading passing rate pt. 3
per_school_passing_math = per_school_passing_math / per_school_count * 100
per_school_passing_reading = per_school_passing_reading / per_school_count *100

In [18]:
# School overall passing rate
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                                 & (school_data_complete_df["math_score"] >= 70)]
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading / per_school_count *100

# School Summary: Formatting

In [19]:
# Set attribute display names
per_school_summary_df = pd.DataFrame({
    "School Type" : per_school_type,
    "Total Students" : per_school_count,
    "Total Budget" : per_school_budget,
    "Per Student Budget" : per_school_capita,
    "Average Math Score" : per_school_math,
    "Average Reading Score" : per_school_reading,
    "% Passing Math" : per_school_passing_math,
    "% Passing Reading" : per_school_passing_reading,
    "% Overall Passing" : per_overall_passing_percentage
})

In [20]:
# format data types
#per_school_summary_df["Total Students"] = per_school_summary_df["Total Students"].map("{:,}".format)
per_school_summary_df["Total Budget"] = per_school_summary_df["Total Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# High-Low Performing School Analysis

In [21]:
# Chart the top 10 schools
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_schools.head(10)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,66.911315,69.663609,65.076453
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887


In [22]:
# Chart the bottom 10 schools
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"])
bottom_schools.head(10)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,66.911315,69.663609,65.076453
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107


In [23]:
# As a result of changing the values Thomas HS dropped from the #2 top school to #7 in overall passing

# Math and Reading Scores by Grade: Attributes

In [24]:
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

ninth_graders_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]


ninth_graders_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

# Math and reading scores by grade: formatting

In [25]:
math_scores_by_grade = pd.DataFrame({
    "9th" : ninth_graders_math_scores,
    "10th" : tenth_graders_math_scores,
    "11th" : eleventh_graders_math_scores,
    "12th" : twelfth_graders_math_scores
})

In [26]:
reading_scores_by_grade = pd.DataFrame({
    "9th" : ninth_graders_reading_scores,
    "10th" : tenth_graders_reading_scores,
    "11th" : eleventh_graders_reading_scores,
    "12th" : twelfth_graders_reading_scores
})

In [27]:
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:,.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:,.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:,.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:,.1f}".format)

math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
math_scores_by_grade.index.name = None

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)

reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Scores by Grade: Analysis

In [28]:
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [29]:
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [30]:
# Despite dropping sigificantly overal as a result of the dropped values
# Thomas HS math and science scores for 10th, 11th, and 12th grades are still among the highest

# Scores by Spending

In [31]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [32]:
# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

In [33]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [34]:
# Assemble into DataFrame. 
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

In [35]:
# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,67,77,56
$645-675,77.0,81.0,66,81,54


In [36]:
# As a result of the change values the % Passing Math, % Passing Reading, and % Overall Passing  averages
# for the $630-644 bracket have decreased by 6, 7, and 7 respectively
# all otehr values remain the same.

# Scores by School Size

In [37]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [38]:
# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

In [39]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]

size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [41]:
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

In [42]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,88,91,85
Large (2000-5000),77.7,81.3,70,83,58


In [43]:
# As a result of the dropped values medium sized schools have had their totals reduced

# Scores By School Type

In [44]:
# Calculate averages for the desired columns. 
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]

type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [45]:
# Assemble into DataFrame.
type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

In [46]:
# Formatting
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,90,93,87
District,77.0,81.0,67,81,54


In [ ]:
# As a result of the nulled values for Thomas HS Charter schools have had their scores drop.